# CNN

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# %cd /content/drive/MyDrive

In [ ]:
# !pip install -q kaggle
# from google.colab import files
# files.upload()
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !cat ~/.kaggle/kaggle.json

In [ ]:
# !kaggle competitions files UBC-OCEAN

In [ ]:
# !kaggle competitions download -c UBC-OCEAN -f train.csv -p /content/drive/MyDrive/Data

In [ ]:
# image_id_array = np.array(df["image_id"])
# competition = "UBC-OCEAN"
# download_path = "/content/training_files_zipped"
# for i in image_id_array:
#     download_command = f"kaggle competitions download -c {competition} -f train_thumbnails/{i}_thumbnail.png -p {download_path}"
#     !download_command

In [ ]:
!pip install keras-cv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from keras import regularizers
from PIL import Image
from numpy import asarray
from keras import layers
from keras.layers import Conv2D
import pickle
import os

os.environ["KERAS_BACKEND"] = "jax"
import keras_cv

sns.set(style="whitegrid")

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=10240)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [ ]:
# !lscpu |grep 'Model name'

In [ ]:
# cpus = tf.config.list_physical_devices('CPU')
# if cpus:
#   # Restrict TensorFlow to only allocate 8GB of memory on the first CPU
#   try:
#     tf.config.set_logical_device_configuration(
#         cpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=8192)])
#     logical_cpus = tf.config.list_logical_devices('CPU')
#     print(len(cpus), "Physical CPUs,", len(logical_cpus), "Logical CPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before CPUs have been initialized
#     print(e)

In [ ]:
class Config:
    is_submission = False

    train_csv_path = "/content/drive/MyDrive/train.csv"
    train_data_paths = "/content/drive/MyDrive/Training_files"
    batch_size = 16
    epochs = 30

    test_csv_path = "/content/drive/MyDrive/test.csv"
    test_data_paths = "/content/drive/MyDrive/Testing_files"

config = Config()

In [ ]:
# img = Image.open("/content/drive/MyDrive/Training_files/10077_thumbnail.png")
# numpydata = asarray(img)

# print(type(numpydata))
# print(numpydata.shape)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/train.csv")

df = df[df["is_tma"] == False]

num_rows = df.shape[0]
num_unique_images = df['image_id'].nunique()
num_unique_labels = df['label'].nunique()
unique_labels = df['label'].unique()

print(f"{num_rows=}")
print(f"{num_unique_images=}")
print(f"{num_unique_labels=}")
print(f"{unique_labels=}")

plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='label', order=df['label'].value_counts().index)
plt.title('Distribution of Target Classes')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
if not config.is_submission:
    # Perform one-hot encoding of the 'label' column and explicitly convert to integer type
    df_one_hot = pd.get_dummies(df["label"], prefix="label").astype(int)

    # Concatenate the original DataFrame with the one-hot encoded labels
    train_df = pd.concat([df["image_id"], df_one_hot], axis=1)

    # Get the thumbnail image paths
    train_df["train_image_path"] = train_df["image_id"].apply(lambda x: f"{config.train_data_paths}/{x}_thumbnail.png")

    train_image_paths = train_df["train_image_path"].values
    labels = train_df[[col for col in train_df.columns if col.startswith("label_")]].values

    label_names = [col for col in train_df.columns if col.startswith("label_")]
    name_to_id = {key.replace("label_", ""):value for value,key in enumerate(label_names)}
    id_to_name = {key:value for value, key in name_to_id.items()}

    # Save to dictionary to disk
    with open("id_to_name.pkl", "wb") as f:
        pickle.dump(id_to_name, f)

In [ ]:
train_df.head()

In [ ]:
def read_image(path):
    file = tf.io.read_file(path)
    image = tf.io.decode_png(file, 0)
    image = tf.image.resize(image, (300, 300), antialias=True)
    image = tf.image.per_image_standardization(image)
    return image

In [ ]:
read_image("/content/drive/MyDrive/Training_files/10077_thumbnail.png").shape

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.25),
  ])

def image_augmenter(path):
    file = tf.io.read_file(path)
    image = tf.io.decode_png(file, 0)
    image = tf.image.resize(image, (300, 300), antialias=True)
    image = data_augmentation(image)
    image = tf.image.per_image_standardization(image)
    return image

In [ ]:
image_augmenter("/content/drive/MyDrive/Training_files/10077_thumbnail.png").shape

In [ ]:
# plt.figure(figsize=(10, 10))
# for i in range(4):
#   augmented_image = image_augmenter("/content/drive/MyDrive/Training_files/10077_thumbnail.png")
#   ax = plt.subplot(2, 2, i + 1)
#   plt.imshow(augmented_image[0])
#   plt.axis("off")

In [ ]:
# if not config.is_submission:
#     class_weights = np.sum(labels) - np.sum(labels, axis=0)
#     class_weights = class_weights / np.sum(class_weights) # Normalize the weights

#     class_weights = {idx:weight for idx, weight in enumerate(class_weights)}

#     for idx, weight in class_weights.items():
#         print(f"{id_to_name[idx]}: {weight:0.2f}")

In [ ]:
label_names

In [ ]:
sample_of_each_image_required = 300
required_augmented_image_dict = {}
intermediate_dict = dict(sample_of_each_image_required - df["label"].value_counts())

for i in [key.replace("label_", "") for key in label_names]:
    required_augmented_image_dict.update({i:intermediate_dict[i]})

print(required_augmented_image_dict)

In [ ]:
# def augmented_image_generator(dictionary):
#     for k,v in dictionary.items():
#         lst = list(train_df[train_df[f"label_{k}"] == 1].sample(v, replace=True)["train_image_path"])
#         tensor = tf.concat([tf.image.per_image_standardization(data_augmentation(read_image(i)))[tf.newaxis,...] for i in lst],0)
#         data = tf.data.Dataset.from_tensors(tensor)
#         data.save(f"/content/augmented_datasets/{k}")

In [ ]:
def images_for_augmentation_paths_generator(dictionary):
    lst_main = []
    for k,v in dictionary.items():
        lst = list(train_df[train_df[f"label_{k}"] == 1].sample(v, replace=True)["train_image_path"])
        lst_main.extend(lst)
    return lst_main

In [ ]:
augmented_images_paths_list = images_for_augmentation_paths_generator(required_augmented_image_dict)

In [ ]:
# def augmented_image_generator(location):
#     data = tf.image.per_image_standardization(data_augmentation(read_image(location)))
#     return data

In [ ]:
augmented_images_labels = []
for i in augmented_images_paths_list:
    new_df = train_df[train_df["train_image_path"] == i]
    new_label = list(new_df[[col for col in new_df.columns if col.startswith("label_")]].values[0])
    augmented_images_labels.append(new_label)

augmented_images_labels_array = np.array(augmented_images_labels)
labels_final = np.concatenate((labels, augmented_images_labels_array), 0)

In [ ]:
if not config.is_submission:
    x1 = (tf.data.Dataset.from_tensor_slices(train_image_paths).map(image_augmenter, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
                                                               .cache("Cached_data_x1")
                                                               .prefetch(buffer_size=tf.data.AUTOTUNE)
          )
    x2 = (tf.data.Dataset.from_tensor_slices(augmented_images_paths_list).map(image_augmenter, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
                                                                         .cache("Cached_data_x2")
                                                                         .prefetch(buffer_size=tf.data.AUTOTUNE)
          )
    x = tf.data.Dataset.concatenate(x1, x2)
    y = tf.data.Dataset.from_tensor_slices(labels_final)

    # Zip the x and y together
    ds = tf.data.Dataset.zip((x, y))
    ds_final = (
        ds
        .shuffle(buffer_size=200)
        .cache("Cached_data_ds")
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )
    # Create the training and validation splits
    val_ds = (
        ds_final
        .take(200)
        .shuffle(100)
        .batch(config.batch_size)
        .cache("Cached_data_val_ds")
        .prefetch(tf.data.AUTOTUNE)
    )
    train_ds = (
        ds_final
        .skip(200)
        .shuffle(100)
        .batch(config.batch_size)
        .cache("Cached_data_train_ds")
        .prefetch(tf.data.AUTOTUNE)
    )

In [ ]:
# if not config.is_submission:
#     images, labels = train_ds.take(1).get_single_element()

#     keras_cv.visualization.plot_image_gallery(
#         images,
#         value_range=(0, 1),
#         rows=2,
#         cols=2,
#     )

In [ ]:
# Pretrained Model
# pretrained_model = keras.applications.EfficientNetV2M(
#     include_top=False,
#     weights="imagenet",
#     input_shape=(1000,1000,3),
#     pooling="avg",
#     classifier_activation="softmax",
#     include_preprocessing=True,
# )

In [ ]:
# pretrained_model =  keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_m_backbone_coco", input_shape=(300,300,3))

In [ ]:
pretrained_model = keras.applications.EfficientNetV2B2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(300,300,3),
    pooling="avg",
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=False,
)

In [ ]:
pretrained_model.trainable = False

model = tf.keras.Sequential()
model.add(pretrained_model)
# model.add(Conv2D(320, (3,3), padding='valid', activation='relu'))
# model.add(Conv2D(160, (3,3), padding='valid', activation='relu'))
# model.add(Conv2D(64, (3,3), padding='valid', activation='relu'))
# model.add(Conv2D(32, (3,3), padding='valid', activation='relu'))
#model.add(Conv2D(16, (3,3), padding='valid', activation='relu'))
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(units=1024, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(units=512, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(units=256, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(units=128, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(units=64, activation="relu"))
model.add(layers.Dense(units=5, activation="softmax"))

model.summary()

In [ ]:
if not config.is_submission:
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("Ovarian_cancer.h5", save_best_only=True)

    history = model.fit(
        train_ds,
        epochs=config.epochs,
        validation_data=val_ds,
        #class_weight=class_weights,
        callbacks=[early_stopping_cb,checkpoint_cb],
        use_multiprocessing=False,
    )

In [ ]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [ ]:
# class Config:
#     is_submission = True

#     train_csv_path = "/content/drive/MyDrive/train.csv"
#     train_data_paths = "/content/drive/MyDrive/Training_files"
#     batch_size = 16
#     epochs = 50

#     test_csv_path = "/content/drive/MyDrive/test.csv"
#     test_data_paths = "/content/drive/MyDrive/Testing_files"

# config = Config()

In [ ]:
# if config.is_submission:
#     df = pd.read_csv(config.test_csv_path)
#     df["image_path"] = df["image_id"].apply(lambda x: f"{config.test_data_paths}/{x}_thumbnail.png")

#     # Load the model weights
#     model.load_weights("/content/drive/MyDrive/Ovarian_cancer.h5")

#     # Load the id to name dictionary
#     with open("/content/id_to_name.pkl", "rb") as f:
#         id_to_name = pickle.load(f)

# Transformer

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Model
from keras.layers import Layer, Embedding, LayerNormalization, Dense, MultiHeadAttention, Flatten, Add
from tensorflow import keras
import seaborn as sns
import pickle
import os

os.environ["KERAS_BACKEND"] = "jax"

sns.set(style="whitegrid")

In [ ]:
class Config:
    is_submission = False

    train_csv_path = "/content/drive/MyDrive/train.csv"
    train_data_paths = "/content/drive/MyDrive/Training_files"
    batch_size = 8

    test_csv_path = "/content/drive/MyDrive/test.csv"
    test_data_paths = "/content/drive/MyDrive/Testing_files"

config = Config()

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/train.csv")

df = df[df["is_tma"] == False]

num_rows = df.shape[0]
num_unique_images = df['image_id'].nunique()
num_unique_labels = df['label'].nunique()
unique_labels = df['label'].unique()

print(f"{num_rows=}")
print(f"{num_unique_images=}")
print(f"{num_unique_labels=}")
print(f"{unique_labels=}")

plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='label', order=df['label'].value_counts().index)
plt.title('Distribution of Target Classes')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
def read_image(path):
    file = tf.io.read_file(path)
    image = tf.io.decode_png(file, 0)
    image = tf.image.resize(image, (500, 300), antialias=True)
    image = tf.image.per_image_standardization(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

In [ ]:
my_img = read_image("/content/drive/MyDrive/Training_files/10077_thumbnail.png")
my_img.shape

In [ ]:
patches = tf.image.extract_patches(
                        images=my_img[tf.newaxis,...],
                        sizes=[1, 21, 21, 1],
                        strides=[1, 21, 21, 1],
                        rates=[1, 2, 2, 1],
                        padding="VALID"
                    )

In [ ]:
patches.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(my_img)
plt.axis("off")

In [ ]:
# Perform one-hot encoding of the 'label' column and explicitly convert to integer type
df_one_hot = pd.get_dummies(df["label"], prefix="label").astype(int)

# Concatenate the original DataFrame with the one-hot encoded labels
train_df = pd.concat([df["image_id"], df_one_hot], axis=1)

# Get the thumbnail image paths
train_df["train_image_path"] = train_df["image_id"].apply(lambda x: f"{config.train_data_paths}/{x}_thumbnail.png")

train_image_paths = train_df["train_image_path"].values
labels = train_df[[col for col in train_df.columns if col.startswith("label_")]].values

label_names = [col for col in train_df.columns if col.startswith("label_")]
name_to_id = {key.replace("label_", ""):value for value,key in enumerate(label_names)}
id_to_name = {key:value for value, key in name_to_id.items()}

# Save to dictionary to disk
with open("id_to_name.pkl", "wb") as f:
    pickle.dump(id_to_name, f)

In [ ]:
class PatchEncoder(Layer):
    def __init__(self, N_PATCHES, HIDDEN_SIZE):
        super(PatchEncoder, self).__init__(name = "patchencoder")

        self.linear_projection = Dense(HIDDEN_SIZE)
        self.positional_embedding = Embedding(N_PATCHES, HIDDEN_SIZE)
        self.N_PATCHES = N_PATCHES

    def call(self, x_train):

        patches = tf.image.extract_patches(
                                images=x_train,
                                sizes=[1, 21, 21, 1],
                                strides=[1, 21, 21, 1],
                                rates=[1, 2, 2, 1],
                                padding="VALID"
                            )

        patches = tf.reshape(patches, (tf.shape(patches)[0], -1, 1323))
        embedding_input = tf.range(start=0, limit=self.N_PATCHES, delta=1)
        output = self.linear_projection(patches) + self.positional_embedding(embedding_input)

        return output

In [ ]:
class TransformerEncoder(Layer):
    def __init__(self, N_HEADS, HIDDEN_SIZE):
        super(TransformerEncoder, self).__init__(name = "transformerencoder")

        self.layer_norm_1 = LayerNormalization()
        self.layer_norm_2 = LayerNormalization()

        self.multi_head_attention = MultiHeadAttention(N_HEADS, HIDDEN_SIZE)

        self.dense_1 = Dense(HIDDEN_SIZE, activation=tf.nn.gelu)
        self.dense_2 = Dense(HIDDEN_SIZE, activation=tf.nn.gelu)

        def call(self, input):
            x_1 = self.layer_norm_1(input)
            x_1 = self.multi_head_attention(x_1, x_1)

            x_1 = Add()([x_1, input])

            x_2 = self.layer_norm_2(x_1)
            x_2 = self.dense_1(x_2)
            x_2 = self.dence_2(x_2)

            output = Add()([x_2, x_1])

            return output

In [ ]:
class VIT(Model):
    def __init__(self, N_HEADS, HIDDEN_SIZE, N_PATCHES, N_LAYERS, N_DENSE, N_CLASSES):
        super(VIT, self).__init__(name = "visiontransformer")

        self.patch_enc = PatchEncoder(N_PATCHES, HIDDEN_SIZE)
        self.transform_encoders = [TransformerEncoder(N_HEADS, HIDDEN_SIZE) for _ in range(N_LAYERS)]
        self.N_LAYERS = N_LAYERS
        self.dense_1 = Dense(N_DENSE, tf.nn.gelu)
        self.dense_2 = Dense(N_DENSE, tf.nn.gelu)
        self.dense_3 = Dense(N_DENSE, tf.nn.gelu)
        self.dense_4 = Dense(N_CLASSES, tf.nn.softmax)

    def call(self, input, training=True):
        x = self.patch_enc(input)

        for i in range(self.N_LAYERS):
            x = self.transform_encoders[i](x)

        x = Flatten()(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.dense_4(x)

        return x

In [ ]:
x = (tf.data.Dataset.from_tensor_slices(train_image_paths).map(read_image, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
                                                          #.cache("Cached_data_x")
                                                          .prefetch(buffer_size=tf.data.AUTOTUNE)
)

y = tf.data.Dataset.from_tensor_slices(labels)

# Zip the x and y together
ds = tf.data.Dataset.zip((x, y))

ds_final = (
    ds
    .shuffle(buffer_size=200)
    #.cache("Cached_data_ds")
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
# Create the training and validation splits
val_ds = (
    ds_final
    .take(100)
    .shuffle(100)
    .batch(config.batch_size)
    #.cache("Cached_data_val_ds")
    .prefetch(tf.data.AUTOTUNE)
)

train_ds = (
    ds_final
    .shuffle(100)
    .batch(config.batch_size)
    #.cache("Cached_data_train_ds")
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
vit_model = VIT(N_HEADS=5, HIDDEN_SIZE=1323, N_PATCHES=286, N_LAYERS=3, N_DENSE=512, N_CLASSES=5)

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=100,
    decay_rate=0.9)

mod_opti = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=True,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adam',
)

In [ ]:
mod_loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    name='categorical_crossentropy'
)

In [ ]:
mod_metrics = ["accuracy"]

In [ ]:
vit_model.compile(
    optimizer=mod_opti,
    loss=mod_loss,
    metrics=mod_metrics,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=True,
    pss_evaluation_shards=0,
)

In [ ]:
history = vit_model.fit(
    train_ds,
    #batch_size=64,
    epochs=10,
    verbose=1,
    #callbacks=[early_stopping_cb, checkpoint_cb],
    #validation_split=0.1,
    validation_data=val_ds,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=6,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=True
)